# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-24-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-23-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-24 05:22:35,33.93911,67.709953,54559,2373,46943,5243.0,Afghanistan,140.152389,4.349420
1,NaN,NaN,NaN,Albania,2021-01-24 05:22:35,41.15330,20.168300,71441,1310,43384,26747.0,Albania,2482.486622,1.833681
2,NaN,NaN,NaN,Algeria,2021-01-24 05:22:35,28.03390,1.659600,105369,2861,71755,30753.0,Algeria,240.288469,2.715220
3,NaN,NaN,NaN,Andorra,2021-01-24 05:22:35,42.50630,1.521800,9499,96,8588,815.0,Andorra,12294.052935,1.010633
4,NaN,NaN,NaN,Angola,2021-01-24 05:22:35,-11.20270,17.873900,19367,457,17249,1661.0,Angola,58.926678,2.359684


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53775,53831,53938,53984,54062,54141,54278,54403,54483,54559
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,65994,66635,67216,67690,67982,68568,69238,69916,70655,71441
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,103127,103381,103611,103833,104092,104341,104606,104852,105124,105369


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2314,2324,2336,2339,2343,2346,2354,2363,2370,2373
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1261,1265,1270,1277,1281,1287,1291,1296,1303,1310
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2822,2827,2831,2836,2840,2843,2849,2853,2856,2861


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,45298,45434,45465,45868,46359,46554,46759,46887,46912,46943
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,39246,39625,40090,40453,40870,41464,41969,42426,42895,43384
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,69992,70188,70373,70554,70747,70933,71127,71343,71552,71755


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5075,5103,5154,5184,5198,5227,5257,5270,5327,5358
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,15841,16002,16176,16251,16346,16513,16653,16798,16981,17128
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1696,1712,1723,1729,1730,1738,1760,1778,1793,1805


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,55,55,55,55,55,55,55,61,61,62
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,177,179,182,182,182,183,184,188,188,189
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,36,36,36,36,36,36,36,36,37,38


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
651,1001.0,Autauga,Alabama,US,2021-01-24 05:22:35,32.539527,-86.644082,5358,62,0,5296.0,"Autauga, Alabama, US",9590.291575,1.157148
688,1075.0,Lamar,Alabama,US,2021-01-24 05:22:35,33.779950,-88.096680,1199,22,0,1177.0,"Lamar, Alabama, US",8685.258964,1.834862
689,1077.0,Lauderdale,Alabama,US,2021-01-24 05:22:35,34.901719,-87.656247,7847,107,0,7740.0,"Lauderdale, Alabama, US",8462.293349,1.363578


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,24994463,417439,0
India,10654533,153339,10316786
Brazil,8816254,216445,7775928


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,24994463,417439,0,24577024,2021-01-24 05:22:35,37.936303,-91.379001
India,10654533,153339,10316786,184408,2021-01-24 05:22:35,23.088275,81.806127
Brazil,8816254,216445,7775928,823881,2021-01-24 05:22:35,-12.669522,-48.480493
Russia,3658447,67919,3076236,514292,2021-01-24 05:22:35,54.546312,62.120860
United Kingdom,3627746,97518,8826,3521402,2021-01-24 05:22:35,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3147735,36846,0
Texas,2242473,34725,0
Florida,1639914,25164,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3147735,36846,0,3110889,2021-01-24 05:22:35,37.843962,-120.728594
Texas,2242473,34725,0,2207748,2021-01-24 05:22:35,31.660643,-98.653069
Florida,1639914,25164,0,1614750,2021-01-24 05:22:35,28.940755,-82.700744
New York,1323312,42134,0,1281178,2021-01-24 05:22:35,42.544151,-75.474183
Illinois,1098527,20645,0,1077882,2021-01-24 05:22:35,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1065505,15184,0
Arizona,Maricopa,444431,6891,0
Illinois,Cook,443600,9227,0
Florida,Miami-Dade,357038,4703,0
Texas,Harris,296521,3978,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1065505,15184,0,1050321,2021-01-24 05:22:35,34.308284,-118.228241,6037.0
Arizona,Maricopa,444431,6891,0,437540,2021-01-24 05:22:35,33.348359,-112.491815,4013.0
Illinois,Cook,443600,9227,0,434373,2021-01-24 05:22:35,41.841448,-87.816588,17031.0
Florida,Miami-Dade,357038,4703,0,352335,2021-01-24 05:22:35,25.611236,-80.551706,12086.0
Texas,Harris,296521,3978,0,292543,2021-01-24 05:22:35,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,54403,69916,104852,9379,19177,192,1843077,165528,28755,399798,...,3553773,34992,78219,1,122260,1546,154063,2115,42213,30047
2021-01-22,54483,70655,105124,9416,19269,195,1853830,165711,28760,401886,...,3594094,36170,78272,1,122795,1548,154557,2118,43333,30523
2021-01-23,54559,71441,105369,9499,19367,195,1862192,165909,28766,403512,...,3627746,36967,78317,1,123245,1548,155006,2118,44592,31007


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,2363,1296,2853,93,448,6,46355,3021,909,7288,...,94765,347,620,0,1129,35,1757,614,597,917
2021-01-22,2370,1303,2856,93,452,6,46575,3030,909,7330,...,96166,364,620,0,1136,35,1769,614,610,962
2021-01-23,2373,1310,2861,96,457,6,46737,3034,909,7389,...,97518,367,621,0,1142,35,1783,615,627,974


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,46887,42426,71343,8474,17176,162,1625755,154046,25961,376360,...,8581,26927,76655,1,114299,1411,142819,1423,31522,19569
2021-01-22,46912,42895,71552,8537,17223,162,1636826,154259,25972,378408,...,8678,27715,76752,1,114925,1411,143611,1423,32667,21080
2021-01-23,46943,43384,71755,8588,17249,162,1646668,154535,25985,380248,...,8826,28547,76778,1,115426,1411,144297,1423,34293,21377


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5075,5103,5154,5184,5198,5227,5257,5270,5327,5358
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1168,1166,1175,1180,1182,1184,1190,1196,1197,1199
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,7402,7451,7528,7546,7600,7631,7686,7733,7798,7847
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2393,2404,2425,2436,2442,2452,2466,2491,2507,2533
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,12120,12261,12393,12465,12515,12603,12749,12901,13036,13137


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-21,432536,52605,0,699942,279220,3100027,381210,235796,72523,49,...,697783,2208871,330469,10580,2305,459604,296087,112617,575688,50424
2021-01-22,436087,52901,0,708041,281382,3124420,383008,237815,73233,49,...,701847,2227789,333118,10759,2321,463751,298249,113615,577991,50583
2021-01-23,439442,53150,0,715357,282995,3147735,384966,237815,73915,49,...,705876,2242473,334889,10908,2341,468655,300198,114752,580003,50583


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-21        5270   16798    1778  2198   5411    1014   1713   11365   
2021-01-22        5327   16981    1793  2212   5439    1022   1724   11441   
2021-01-23        5358   17128    1805  2223   5462    1033   1731   11496   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-21         2973     1625  ...    2508    542     2811      639   
2021-01-22         3011     1643  ...    2507    540     2824      640   
2021-01-23         3034     1651  ...    2507    540     2824      640   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-21           3370  2847  1874          0      848    609  
2021-01-22           3384  2870  1887          0      853    609  
2021-01-23           3384  2870  1887          0      853    609  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-21,6379,254,0,11772,4496,35796,5440,6774,1027,0,...,8684,33942,1547,168,24,5940,4065,1849,6090,550
2021-01-22,6486,254,0,12001,4549,36362,5462,6819,1027,0,...,8777,34381,1571,169,24,6002,4114,1856,6135,571
2021-01-23,6657,259,0,12170,4563,36846,5482,6819,1036,0,...,8819,34725,1582,170,24,6079,4114,1872,6184,571


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-21          61     188      36   48     86      28     58     205   
2021-01-22          61     188      37   48     86      28     59     206   
2021-01-23          62     189      38   48     86      28     60     206   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-21           70       30  ...      17     10       18        7   
2021-01-22           70       30  ...      18     10       18        7   
2021-01-23           70       31  ...      18     10       18        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-21             28     5    10          0       24      4  
2021-01-22             30     5    12          0       25      4  
2021-01-23             30     5    12          0       25      4  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,0.002303,0.009792,0.002352,0.007628,0.004400,0.010526,0.006222,0.001858,0.000209,0.004275,...,0.010802,0.020353,0.000716,0.0,0.004676,0.001295,0.003080,0.000000,0.030868,0.021729
2021-01-22,0.001471,0.010570,0.002594,0.003945,0.004797,0.015625,0.005834,0.001106,0.000174,0.005223,...,0.011346,0.033665,0.000678,0.0,0.004376,0.001294,0.003206,0.001418,0.026532,0.015842
2021-01-23,0.001395,0.011124,0.002331,0.008815,0.005086,0.000000,0.004511,0.001195,0.000209,0.004046,...,0.009363,0.022035,0.000575,0.0,0.003665,0.000000,0.002905,0.000000,0.029054,0.015857


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,0.003823,0.003873,0.001404,0.010870,0.009009,0.0,0.003008,0.001658,0.0,0.007047,...,0.013866,0.032738,0.000000,NaN,0.006239,0.0,0.003427,0.003268,0.020513,0.043231
2021-01-22,0.002962,0.005401,0.001052,0.000000,0.008929,0.0,0.004746,0.002979,0.0,0.005763,...,0.014784,0.048991,0.000000,NaN,0.006200,0.0,0.006830,0.000000,0.021776,0.049073
2021-01-23,0.001266,0.005372,0.001751,0.032258,0.011062,0.0,0.003478,0.001320,0.0,0.008049,...,0.014059,0.008242,0.001613,NaN,0.005282,0.0,0.007914,0.001629,0.027869,0.012474


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,0.002737,0.010889,0.003037,0.008930,0.015070,0.031847,0.007425,0.001228,0.000347,0.004098,...,0.010480,0.030975,0.000405,0.0,0.002781,0.003556,0.005739,0.0,0.058673,0.016413
2021-01-22,0.000533,0.011055,0.002930,0.007435,0.002736,0.000000,0.006810,0.001383,0.000424,0.005442,...,0.011304,0.029264,0.001265,0.0,0.005477,0.000000,0.005545,0.0,0.036324,0.077214
2021-01-23,0.000661,0.011400,0.002837,0.005974,0.001510,0.000000,0.006013,0.001789,0.000501,0.004862,...,0.017055,0.030020,0.000339,0.0,0.004359,0.000000,0.004777,0.0,0.049775,0.014089


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-21,0.006705,0.004046,NaN,0.013610,0.011249,0.008201,0.005229,0.007098,0.010421,0.0,...,0.005030,0.010669,0.006362,0.010410,0.000000,0.008808,0.007040,0.008417,0.004482,0.005985
2021-01-22,0.008210,0.005627,NaN,0.011571,0.007743,0.007869,0.004717,0.008562,0.009790,0.0,...,0.005824,0.008565,0.008016,0.016919,0.006941,0.009023,0.007302,0.008862,0.004000,0.003153
2021-01-23,0.007693,0.004707,NaN,0.010333,0.005732,0.007462,0.005112,0.000000,0.009313,0.0,...,0.005741,0.006591,0.005316,0.013849,0.008617,0.010575,0.006535,0.010007,0.003481,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-21      0.002473  0.008707  0.010227  0.004570  0.006510  0.002967   
2021-01-22      0.010816  0.010894  0.008436  0.006369  0.005175  0.007890   
2021-01-23      0.005819  0.008657  0.006693  0.004973  0.004229  0.010763   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-21      0.002927  0.006465  0.014330  0.003706  ...  0.008444   
2021-01-22      0.006421  0.006687  0.012782  0.011077  ... -0.000399   
2021-01-23      0.004060  0.004807  0.007639  0.004869  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-21      0.003704  0.001425  0.004717   0.006872  0.021162  0.011879   
2021-01-22     -0.003690  0.004625  0.001565   0.004154  0.008079  0.006937   
2021-01-23      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   

Province_State                                
Admin2         Unassigned  Washakie   Weston  
2021-01-21            NaN  0.002364  0.00495  
2021-01-22            NaN  0.005896  0.00000  
2021-01-23            NaN  0.000000  0.00000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-21,0.015279,0.003953,NaN,0.021166,0.012385,0.020993,0.003320,0.007136,0.000975,NaN,...,0.014960,0.013466,0.019776,0.018182,0.0,0.013479,0.031726,0.007081,0.009114,0.000000
2021-01-22,0.016774,0.000000,NaN,0.019453,0.011788,0.015812,0.004044,0.006643,0.000000,NaN,...,0.010709,0.012934,0.015514,0.005952,0.0,0.010438,0.012054,0.003786,0.007389,0.038182
2021-01-23,0.026364,0.019685,NaN,0.014082,0.003078,0.013311,0.003662,0.000000,0.008763,NaN,...,0.004785,0.010006,0.007002,0.005917,0.0,0.012829,0.000000,0.008621,0.007987,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-01-21      0.109091  0.021739  0.000000  0.021277  0.036145     0.0   
2021-01-22      0.000000  0.000000  0.027778  0.000000  0.000000     0.0   
2021-01-23      0.016393  0.005319  0.027027  0.000000  0.000000     0.0   

Province_State                                         ...   Wyoming         \
Admin2            Butler   Calhoun Chambers  Cherokee  ...      Park Platte   
2021-01-21      0.000000  0.019900      0.0  0.000000  ...  0.000000    0.0   
2021-01-22      0.017241  0.004878      0.0  0.000000  ...  0.058824    0.0   
2021-01-23      0.016949  0.000000      0.0  0.033333  ...  0.000000    0.0   

Province_State                                                                \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned  Washakie   
2021-01-21          0.0      0.0   0.000000   0.0   0.0        NaN  0.000000   
2021-01-22          0.0      0.0   0.071429   0.0   0.2        NaN  0.041667   
2021-01-23          0.0      0.0   0.000000   0.0   0.0        NaN  0.000000   

Province_State         
Admin2         Weston  
2021-01-21        0.0  
2021-01-22        0.0  
2021-01-23        0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,0.002139,0.009197,0.002406,0.008893,0.004348,0.006704,0.006215,0.001742,0.000277,0.004100,...,0.010916,0.021301,0.000763,0.0,0.004809,0.001526,0.003220,0.000164,0.032661,0.023759
2021-01-22,0.001805,0.009883,0.002500,0.006419,0.004573,0.011165,0.006025,0.001424,0.000225,0.004661,...,0.011131,0.027483,0.000720,0.0,0.004592,0.001410,0.003213,0.000791,0.029597,0.019800
2021-01-23,0.001600,0.010504,0.002415,0.007617,0.004829,0.005582,0.005268,0.001309,0.000217,0.004354,...,0.010247,0.024759,0.000647,0.0,0.004128,0.000705,0.003059,0.000396,0.029325,0.017829


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,0.003218,0.003784,0.001550,0.007422,0.007476,0.000781,0.003685,0.002255,1.641097e-11,0.006958,...,0.015279,0.029047,0.000218,NaN,0.005559,1.426689e-44,0.005151,0.001635,0.020197,0.054318
2021-01-22,0.003090,0.004593,0.001301,0.003711,0.008202,0.000391,0.004215,0.002617,8.205485e-12,0.006360,...,0.015032,0.039019,0.000109,NaN,0.005880,7.133444e-45,0.005990,0.000818,0.020986,0.051695
2021-01-23,0.002178,0.004982,0.001526,0.017984,0.009632,0.000195,0.003847,0.001969,4.102743e-12,0.007205,...,0.014545,0.023631,0.000861,NaN,0.005581,3.566722e-45,0.006952,0.001223,0.024427,0.032085


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-21,0.004040,0.011574,0.002867,0.009193,0.011957,0.016830,0.006811,0.001790,0.000369,0.004579,...,0.013781,0.030433,0.000518,0.0,0.002852,0.003563,0.006372,0.000359,0.053483,0.035031
2021-01-22,0.002287,0.011314,0.002898,0.008314,0.007347,0.008415,0.006811,0.001587,0.000396,0.005010,...,0.012543,0.029849,0.000892,0.0,0.004164,0.001781,0.005959,0.000180,0.044904,0.056123
2021-01-23,0.001474,0.011357,0.002868,0.007144,0.004428,0.004207,0.006412,0.001688,0.000448,0.004936,...,0.014799,0.029934,0.000615,0.0,0.004262,0.000891,0.005368,0.000090,0.047339,0.035106


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-21,0.006511,0.003659,NaN,0.010988,0.009224,0.008498,0.004587,0.008775,0.008823,8.160062e-95,...,0.005125,0.010588,0.005947,0.012102,0.005049,0.010361,0.006708,0.008459,0.003855,0.005315
2021-01-22,0.007360,0.004643,NaN,0.011280,0.008484,0.008183,0.004652,0.008669,0.009306,4.080031e-95,...,0.005475,0.009576,0.006981,0.014510,0.005995,0.009692,0.007005,0.008661,0.003928,0.004234
2021-01-23,0.007527,0.004675,NaN,0.010806,0.007108,0.007823,0.004882,0.004334,0.009310,2.040016e-95,...,0.005608,0.008084,0.006149,0.014180,0.007306,0.010133,0.006770,0.009334,0.003704,0.002117


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-21      0.004011  0.008578  0.009223  0.005455  0.006503  0.006118   
2021-01-22      0.007414  0.009736  0.008830  0.005912  0.005839  0.007004   
2021-01-23      0.006617  0.009196  0.007761  0.005443  0.005034  0.008884   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-21      0.002571  0.006295  0.013798  0.004016  ...  0.007769   
2021-01-22      0.004496  0.006491  0.013290  0.007546  ...  0.003685   
2021-01-23      0.004278  0.005649  0.010464  0.006208  ...  0.001842   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-21      0.005391  0.001924  0.003212   0.006112  0.015754  0.008655   
2021-01-22      0.000850  0.003274  0.002388   0.005133  0.011917  0.007796   
2021-01-23      0.000425  0.001637  0.001194   0.002566  0.005958  0.003898   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-21           -1.0  0.001719  0.003546  
2021-01-22           -1.0  0.003808  0.001773  
2021-01-23           -1.0  0.001904  0.000887  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-21,0.014746,0.027605,NaN,0.017021,0.011423,0.018506,0.003605,0.006258,0.002625,NaN,...,0.011282,0.012167,0.012883,0.012320,1.658564e-07,0.011253,0.017192,0.009698,0.008845,0.006773
2021-01-22,0.015760,0.013803,NaN,0.018237,0.011606,0.017159,0.003824,0.006451,0.001312,NaN,...,0.010996,0.012551,0.014199,0.009136,8.292820e-08,0.010846,0.014623,0.006742,0.008117,0.022477
2021-01-23,0.021062,0.016744,NaN,0.016160,0.007342,0.015235,0.003743,0.003225,0.005038,NaN,...,0.007890,0.011278,0.010600,0.007527,4.146410e-08,0.011837,0.007312,0.007681,0.008052,0.011239


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-21      0.054566  0.013330  0.000113  0.010558  0.018424  0.000017   
2021-01-22      0.027283  0.006665  0.013945  0.005279  0.009212  0.000008   
2021-01-23      0.021838  0.005992  0.020486  0.002639  0.004606  0.000004   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-21      0.000102  0.011454  0.003812  0.000338  ...  0.007890   
2021-01-22      0.008672  0.008166  0.001906  0.000169  ...  0.033357   
2021-01-23      0.012810  0.004083  0.000953  0.016751  ...  0.016678   

Province_State                                                         \
Admin2            Platte  Sheridan      Sublette Sweetwater     Teton   
2021-01-21      0.013890  0.007353  1.319250e-06   0.009706  0.031250   
2021-01-22      0.006945  0.003676  6.596248e-07   0.040567  0.015625   
2021-01-23      0.003472  0.001838  3.298124e-07   0.020284  0.007813   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2021-01-21      0.014015  -0.501961  0.005529  0.000014  
2021-01-22      0.107008  -0.501961  0.023598  0.000007  
2021-01-23      0.053504  -0.501961  0.011799  0.000003  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210124,AK,51610,NaN,1401860.0,NaN,totalTestsViral,1453470,57.0,1185.0,...,1453470,0,1,657bf064457b28ae60abcd8b2ac6ccf4b4c11643,0,0,0,0,0,NaN
1,20210124,AL,441170,91709.0,1737094.0,NaN,totalTestsPeopleViral,2086555,2254.0,40514.0,...,2178264,3,0,a5886e16177a22e9e711e0d36eafcaecba90307d,0,0,0,0,0,NaN
2,20210124,AR,284066,56238.0,2119560.0,NaN,totalTestsViral,2347388,1080.0,13267.0,...,2403626,43,17,dcd628b60b2155bbe3365ea432671d94b2e69596,0,0,0,0,0,NaN
3,20210124,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,3c81ee980bf91ef3fe383f82e1094d0d8fc76e2c,0,0,0,0,0,NaN
4,20210124,AZ,722574,44099.0,2671771.0,NaN,totalTestsViral,6385219,4320.0,50264.0,...,3394345,68,301,2453c8080bd6d93c891880b4084cd6e120bbad71,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210124,AK,51610.0,NaN,1401860.0,NaN,totalTestsViral,1453470.0,57.0,1185.0,...,1453470,0,1,657bf064457b28ae60abcd8b2ac6ccf4b4c11643,0,0,0,0,0,NaN
1,20210124,AL,441170.0,91709.0,1737094.0,NaN,totalTestsPeopleViral,2086555.0,2254.0,40514.0,...,2178264,3,0,a5886e16177a22e9e711e0d36eafcaecba90307d,0,0,0,0,0,NaN
2,20210124,AR,284066.0,56238.0,2119560.0,NaN,totalTestsViral,2347388.0,1080.0,13267.0,...,2403626,43,17,dcd628b60b2155bbe3365ea432671d94b2e69596,0,0,0,0,0,NaN
3,20210124,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,3c81ee980bf91ef3fe383f82e1094d0d8fc76e2c,0,0,0,0,0,NaN
4,20210124,AZ,722574.0,44099.0,2671771.0,NaN,totalTestsViral,6385219.0,4320.0,50264.0,...,3394345,68,301,2453c8080bd6d93c891880b4084cd6e120bbad71,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-24,AK,51610,NaN,1401860.0,NaN,totalTestsViral,1453470,57.0,1185.0,NaN,...,1453470,0,1,657bf064457b28ae60abcd8b2ac6ccf4b4c11643,0,0,0,0,0,NaN
2021-01-24,AL,441170,91709.0,1737094.0,NaN,totalTestsPeopleViral,2086555,2254.0,40514.0,NaN,...,2178264,3,0,a5886e16177a22e9e711e0d36eafcaecba90307d,0,0,0,0,0,NaN
2021-01-24,AR,284066,56238.0,2119560.0,NaN,totalTestsViral,2347388,1080.0,13267.0,345.0,...,2403626,43,17,dcd628b60b2155bbe3365ea432671d94b2e69596,0,0,0,0,0,NaN
2021-01-24,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,3c81ee980bf91ef3fe383f82e1094d0d8fc76e2c,0,0,0,0,0,NaN
2021-01-24,AZ,722574,44099.0,2671771.0,NaN,totalTestsViral,6385219,4320.0,50264.0,1054.0,...,3394345,68,301,2453c8080bd6d93c891880b4084cd6e120bbad71,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-24,AK,51610.0,NaN,1401860.0,NaN,totalTestsViral,1453470.0,57.0,1185.0,NaN,...,1453470,0,1,657bf064457b28ae60abcd8b2ac6ccf4b4c11643,0,0,0,0,0,NaN
2021-01-24,AL,441170.0,91709.0,1737094.0,NaN,totalTestsPeopleViral,2086555.0,2254.0,40514.0,NaN,...,2178264,3,0,a5886e16177a22e9e711e0d36eafcaecba90307d,0,0,0,0,0,NaN
2021-01-24,AR,284066.0,56238.0,2119560.0,NaN,totalTestsViral,2347388.0,1080.0,13267.0,345.0,...,2403626,43,17,dcd628b60b2155bbe3365ea432671d94b2e69596,0,0,0,0,0,NaN
2021-01-24,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,3c81ee980bf91ef3fe383f82e1094d0d8fc76e2c,0,0,0,0,0,NaN
2021-01-24,AZ,722574.0,44099.0,2671771.0,NaN,totalTestsViral,6385219.0,4320.0,50264.0,1054.0,...,3394345,68,301,2453c8080bd6d93c891880b4084cd6e120bbad71,0,0,0,0,0,NaN
